In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

import warnings
from sklearn.exceptions import DataConversionWarning

# Ignore all warnings from scikit-learn
warnings.filterwarnings("ignore", module="sklearn")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
# Add more categories if needed



In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 5,
    "exp_name": ct.generate_readable_short_id("test"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,

    "calib_methods": ["RF_d", "RF_opt", "RF_large",
                      "Platt", "ISO", "Beta", "PPA", "VA",
                      "Rank"                   
                      ],

    # "calib_methods": ["RF_opt", "RF_large",
    #                   "DT", "LR", "SVM", "NN", "GNB"          
    #                   ],

    "metrics": ["acc", "brier", "ece", "logloss", "time"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [50],
                    "max_depth":  np.arange(2, 100).tolist(), # [None, 5, 10, 15, 20, 30, 50], #
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": [2, 5, 10],
                    "min_samples_leaf":  [1, 2, 4],
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "data_name",
    "exp_values": [
                  "cm1",
                  "datatrieve",
                  "kc1_class_level_defectiveornot",
                  "kc1",
                  "kc2",
                  "kc3",
                  "pc1",
                  "spect",
                  "spectf",
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                  "bank", 
                  "ionosphere", 
                  "HRCompetencyScores",
                  "spambase", 
                  "QSAR", 
                  "diabetes", 
                  "breast", 
                  "SPF",
                  "hillvalley",
                  "pc4",
                  "scene",
                  "Sonar_Mine_Rock_Data",
                  "Customer_Churn",
                  "jm1",
                  "eeg",
                  "phoneme",


                  # "madelon",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param cm1 done
exp_param datatrieve done
exp_param kc1_class_level_defectiveornot done
exp_param kc1 done
exp_param kc2 done
exp_param kc3 done
exp_param pc1 done
exp_param spect done
exp_param spectf done
exp_param vertebral done
exp_param wilt done
exp_param parkinsons done
exp_param heart done
exp_param wdbc done
exp_param bank done
exp_param ionosphere done
exp_param HRCompetencyScores done
exp_param spambase done
exp_param QSAR done
exp_param diabetes done
exp_param breast done
exp_param SPF done
exp_param hillvalley done
exp_param pc4 done
exp_param scene done
exp_param Sonar_Mine_Rock_Data done
exp_param Customer_Churn done
exp_param jm1 done
exp_param eeg done
exp_param phoneme done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
tables["time"].round(4)


,RF_d,RF_opt,RF_large,Platt,ISO,Beta,PPA,VA,Rank
Data,,,,,,,,,
cm1,0.0510,20.4754,0.2224,17.3045,17.3042,17.3066,17.8086,17.3038,17.3904
datatrieve,0.0310,10.7910,0.1498,10.3679,10.3675,10.3700,10.8079,10.3672,10.4035
kc1_class_level_defectiveornot,0.0368,15.4850,0.1713,13.6507,13.6499,13.6521,14.0782,13.6496,13.6843
kc1,0.1145,58.4244,0.5217,45.7403,45.7392,45.7406,46.4353,45.7389,46.0608
kc2,0.0453,19.9276,0.2254,16.8760,16.8755,16.8772,17.3643,16.8752,16.9633
kc3,0.0482,20.0385,0.2135,16.8430,16.8425,16.8441,17.3139,16.8423,16.9231
pc1,0.0746,34.9961,0.3495,27.9302,27.9298,27.9312,28.5098,27.9294,28.1063
spect,0.0378,12.5167,0.1639,11.7333,11.7327,11.7337,12.2150,11.7323,11.7885
spectf,0.0473,17.6739,0.2002,15.3433,15.3428,15.3439,15.8022,15.3420,15.4041


In [6]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}")

metric acc
Friedman Test Statistic: 100.93959731543623
P-value: 2.7431464636094317e-18
The differences between groups are significant.
metric brier
Friedman Test Statistic: 129.40444444444438
P-value: 3.7590285285854896e-24
The differences between groups are significant.
metric ece
Friedman Test Statistic: 37.21777777777777
P-value: 1.0494446305323842e-05
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 143.85777777777776
P-value: 3.736587372022943e-27
The differences between groups are significant.
metric time
Friedman Test Statistic: 239.21777777777788
P-value: 3.3158610303011354e-47
The differences between groups are significant.


In [7]:
ct.save_metrics_to_latex(tables, params["metrics"], params['exp_name'])